In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from functools import reduce

# Paths
base_path = r'C:\Users\karun\OneDrive\Documents\RIK\outputs'
final_output_path = r'C:\Users\karun\OneDrive\Documents\RIK\outputs\final_outputs\final_processed_data.csv'

# List of files to process
cert_files = [
    'daily_user_devices_enriched.csv',
    'daily_user_email_enriched.csv',
    'daily_user_files_enriched.csv',
    'daily_user_http_enriched.csv',
    'daily_user_logons_enriched.csv',
    'RIK_final_training_data.csv'
]
twos_files = [
    'twos_emails_liwc.csv',
    'twos_eventviewer_summary.csv',
    'twos_keystroke_summary.csv',
    'twos_mouse_summary.csv',
    'twos_network_summary.csv',
    'twos_personality_summary.csv'
]

all_files = cert_files + twos_files

# Columns to encode and drop
encode_cols = ['user', 'role', 'business_unit', 'functional_unit', 'department', 'team']
drop_cols = ['employee_name', 'email', 'projects', 'supervisor', 'user_id']

# Helper: Extract date features
def extract_date_features(df, date_col='date_only'):
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df['day_of_week'] = df[date_col].dt.dayofweek
        df['month'] = df[date_col].dt.month
        df['day'] = df[date_col].dt.day
        df = df.drop(columns=[date_col])
    return df

# Read, process, and merge all files
dfs = []
le_dict = {}

for file in all_files:
    path = f"{base_path}\\{file}"
    df = pd.read_csv(path, low_memory=False, dtype=str)  # Read all as string to avoid dtype issues

    # Encode categorical columns
    for col in encode_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            le_dict[col] = le

    # Drop irrelevant columns
    df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

    # Extract date features
    df = extract_date_features(df)

    # Convert all columns except encoded ones to numeric (if possible)
    for col in df.columns:
        if col not in encode_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Ensure merge keys exist and have consistent types
    for key in ['user', 'day_of_week', 'month', 'day']:
        if key not in df.columns:
            df[key] = pd.NA
        df[key] = pd.to_numeric(df[key], errors='coerce').astype('Int64')

    # Keep only columns with at least 20% non-missing values (feature coverage filter)
    feature_coverage = df.notnull().mean()
    keep_cols = [col for col in df.columns if feature_coverage[col] > 0.2 or col in ['user', 'day_of_week', 'month', 'day']]
    df = df[keep_cols]

    # Only keep numeric columns and merge keys
    keep_numeric = [col for col in df.columns if df[col].dtype in ['int64', 'float64', 'Int64'] or col in ['user', 'day_of_week', 'month', 'day']]
    df = df[keep_numeric]

    dfs.append(df)

# Merge all DataFrames on merge keys
merge_keys = ['user', 'day_of_week', 'month', 'day']
def merge_dfs(dfs, keys):
    return reduce(lambda left, right: pd.merge(left, right, on=keys, how='outer'), dfs)

final_df = merge_dfs(dfs, merge_keys)

# Drop columns with more than 90% missing values after merge
missing_thresh = 0.9
final_df = final_df.loc[:, final_df.isnull().mean() < missing_thresh]

# Fill missing values: float columns with mean, integer columns with median
for col in final_df.columns:
    if pd.api.types.is_float_dtype(final_df[col]):
        final_df[col] = final_df[col].fillna(final_df[col].mean())
    elif pd.api.types.is_integer_dtype(final_df[col]):
        final_df[col] = final_df[col].fillna(final_df[col].median())

# Save final processed data
final_df.to_csv(final_output_path, index=False)

print(f"Final processed data saved to: {final_output_path}")

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from functools import reduce

# Paths
base_path = r'C:\Users\karun\OneDrive\Documents\RIK\outputs'
final_output_path = r'C:\Users\karun\OneDrive\Documents\RIK\outputs\final_outputs\final_processed_data.csv'

# List of files to process
cert_files = [
    'daily_user_devices_enriched.csv',
    'daily_user_email_enriched.csv',
    'daily_user_files_enriched.csv',
    'daily_user_http_enriched.csv',
    'daily_user_logons_enriched.csv',
    'RIK_final_training_data.csv'
]
twos_files = [
    'twos_emails_liwc.csv',
    'twos_eventviewer_summary.csv',
    'twos_keystroke_summary.csv',
    'twos_mouse_summary.csv',
    'twos_network_summary.csv',
    'twos_personality_summary.csv'
]

all_files = cert_files + twos_files

# Columns to encode and drop
encode_cols = ['user', 'role', 'business_unit', 'functional_unit', 'department', 'team']
drop_cols = ['employee_name', 'email', 'projects', 'supervisor', 'user_id']

# Helper: Extract date features
def extract_date_features(df, date_col='date_only'):
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df['day_of_week'] = df[date_col].dt.dayofweek
        df['month'] = df[date_col].dt.month
        df['day'] = df[date_col].dt.day
        df = df.drop(columns=[date_col])
    return df

# Read, process, and merge all files
dfs = []
le_dict = {}

for file in all_files:
    path = f"{base_path}\\{file}"
    df = pd.read_csv(path, low_memory=False, dtype=str)  # Read all as string to avoid dtype issues

    # Encode categorical columns
    for col in encode_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            le_dict[col] = le

    # Drop irrelevant columns
    df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

    # Extract date features
    df = extract_date_features(df)

    # Convert all columns except encoded ones to numeric (if possible)
    for col in df.columns:
        if col not in encode_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Ensure merge keys exist and have consistent types
    for key in ['user', 'day_of_week', 'month', 'day']:
        if key not in df.columns:
            df[key] = pd.NA
        # Convert merge keys to int (except 'user' which is already encoded as int)
        if key != 'user':
            df[key] = pd.to_numeric(df[key], errors='coerce').astype('Int64')
        else:
            df[key] = pd.to_numeric(df[key], errors='coerce').astype('Int64')

    # Keep only numeric columns and merge keys
    keep_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64', 'Int64'] or col in ['user', 'day_of_week', 'month', 'day']]
    df = df[keep_cols]

    # Handle missing values (fill with column mean)
    df = df.fillna(df.mean(numeric_only=True))

    dfs.append(df)

# Merge all DataFrames on merge keys
merge_keys = ['user', 'day_of_week', 'month', 'day']
def merge_dfs(dfs, keys):
    return reduce(lambda left, right: pd.merge(left, right, on=keys, how='outer'), dfs)

final_df = merge_dfs(dfs, merge_keys)

# Drop columns with more than 90% missing values after merge
missing_thresh = 0.9
final_df = final_df.loc[:, final_df.isnull().mean() < missing_thresh]

# Fill missing values: float columns with mean, integer columns with median
for col in final_df.columns:
    if pd.api.types.is_float_dtype(final_df[col]):
        final_df[col] = final_df[col].fillna(final_df[col].mean())
    elif pd.api.types.is_integer_dtype(final_df[col]):
        final_df[col] = final_df[col].fillna(final_df[col].median())

# Save final processed data
final_df.to_csv(final_output_path, index=False)

print(f"Final processed data saved to: {final_output_path}")

Final processed data saved to: C:\Users\karun\OneDrive\Documents\RIK\outputs\final_outputs\final_processed_data.csv


In [6]:
import pandas as pd

df = pd.read_csv(r'C:\Users\karun\OneDrive\Documents\RIK\outputs\final_outputs\final_processed_data.csv')

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Missing values per column:\n", df.isnull().sum())
print("Sample data:\n", df.head())

Shape: (1464618, 219)
Columns: ['user', 'device_events_per_day_x', 'unique_pcs_per_day_x_x', 'unique_filetrees_per_day_x', 'connect_events_per_day_x', 'disconnect_events_per_day_x', 'external_path_access_per_day_x', 'day_of_week', 'month', 'day', 'emails_sent_per_day_x', 'emails_viewed_per_day_x', 'avg_email_size_per_day_x', 'attachments_per_day_x', 'multi_recipient_emails_per_day_x', 'outbound_emails_per_day_x', 'unique_recipients_per_day_x', 'cc_recipients_per_day_x', 'bcc_recipients_per_day_x', 'avg_attachment_size_per_day_x', 'file_events_per_day_x', 'unique_files_per_day_x', 'usb_copies_per_day_x', 'usb_reads_per_day_x', 'file_write_events_x', 'file_open_events_x', 'decoy_file_access_per_day_x', 'web_visits_per_day_x', 'unique_domains_per_day_x', 'unique_paths_per_day_x', 'suspicious_domains_per_day_x', 'file_hosting_visits_per_day_x', 'suspicious_protocol_visits_per_day_x', 'visits_out_of_hours_per_day_x', 'logins_per_day_x', 'unique_pcs_per_day_y_x', 'logins_out_of_hours_x', 'we

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from functools import reduce

# Paths
base_path = r'C:\Users\karun\OneDrive\Documents\RIK\outputs'
final_output_path = r'C:\Users\karun\OneDrive\Documents\RIK\outputs\final_outputs\final_processed_data.csv'

# List of files to process
cert_files = [
    'daily_user_devices_enriched.csv',
    'daily_user_email_enriched.csv',
    'daily_user_files_enriched.csv',
    'daily_user_http_enriched.csv',
    'daily_user_logons_enriched.csv',
    'RIK_final_training_data.csv'
]
twos_files = [
    'twos_emails_liwc.csv',
    'twos_eventviewer_summary.csv',
    'twos_keystroke_summary.csv',
    'twos_mouse_summary.csv',
    'twos_network_summary.csv',
    'twos_personality_summary.csv'
]

all_files = cert_files + twos_files

# Columns to encode and drop
encode_cols = ['user', 'role', 'business_unit', 'functional_unit', 'department', 'team']
drop_cols = ['employee_name', 'email', 'projects', 'supervisor', 'user_id']

# Helper: Extract date features
def extract_date_features(df, date_col='date_only'):
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df['day_of_week'] = df[date_col].dt.dayofweek
        df['month'] = df[date_col].dt.month
        df['day'] = df[date_col].dt.day
        df = df.drop(columns=[date_col])
    return df

# Read, process, and merge all files
dfs = []
le_dict = {}

for file in all_files:
    path = f"{base_path}\\{file}"
    df = pd.read_csv(path, low_memory=False, dtype=str)  # Read all as string to avoid dtype issues

    # Encode categorical columns
    for col in encode_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            le_dict[col] = le

    # Drop irrelevant columns
    df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

    # Extract date features
    df = extract_date_features(df)

    # Convert all columns except encoded ones to numeric (if possible)
    for col in df.columns:
        if col not in encode_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Ensure merge keys exist and have consistent types
    for key in ['user', 'day_of_week', 'month', 'day']:
        if key not in df.columns:
            df[key] = pd.NA
        df[key] = pd.to_numeric(df[key], errors='coerce').astype('Int64')

    # Keep only columns with at least 20% non-missing values (feature coverage filter)
    feature_coverage = df.notnull().mean()
    keep_cols = [col for col in df.columns if feature_coverage[col] > 0.2 or col in ['user', 'day_of_week', 'month', 'day']]
    df = df[keep_cols]

    # Only keep numeric columns and merge keys
    keep_numeric = [col for col in df.columns if df[col].dtype in ['int64', 'float64', 'Int64'] or col in ['user', 'day_of_week', 'month', 'day']]
    df = df[keep_numeric]

    dfs.append(df)

# Merge all DataFrames on merge keys
merge_keys = ['user', 'day_of_week', 'month', 'day']
def merge_dfs(dfs, keys):
    return reduce(lambda left, right: pd.merge(left, right, on=keys, how='outer'), dfs)

final_df = merge_dfs(dfs, merge_keys)

# Drop columns with more than 90% missing values after merge
missing_thresh = 0.9
final_df = final_df.loc[:, final_df.isnull().mean() < missing_thresh]

# Fill missing values: float columns with mean, integer columns with median
for col in final_df.columns:
    if pd.api.types.is_float_dtype(final_df[col]):
        final_df[col] = final_df[col].fillna(final_df[col].mean())
    elif pd.api.types.is_integer_dtype(final_df[col]):
        final_df[col] = final_df[col].fillna(final_df[col].median())

# Save final processed data
final_df.to_csv(final_output_path, index=False)

print(f"Final processed data saved to: {final_output_path}")

Final processed data saved to: C:\Users\karun\OneDrive\Documents\RIK\outputs\final_outputs\final_processed_data.csv


In [9]:
import pandas as pd

df = pd.read_csv(r'C:\Users\karun\OneDrive\Documents\RIK\outputs\final_outputs\final_processed_data.csv')

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Missing values per column:\n", df.isnull().sum())
print("Sample data:\n", df.head())

Shape: (1464618, 93)
Columns: ['user', 'device_events_per_day_x', 'unique_pcs_per_day_x_x', 'unique_filetrees_per_day_x', 'connect_events_per_day_x', 'disconnect_events_per_day_x', 'external_path_access_per_day_x', 'day_of_week', 'month', 'day', 'emails_sent_per_day_x', 'emails_viewed_per_day_x', 'avg_email_size_per_day_x', 'attachments_per_day_x', 'multi_recipient_emails_per_day_x', 'outbound_emails_per_day_x', 'unique_recipients_per_day_x', 'cc_recipients_per_day_x', 'bcc_recipients_per_day_x', 'avg_attachment_size_per_day_x', 'file_events_per_day_x', 'unique_files_per_day_x', 'usb_copies_per_day_x', 'usb_reads_per_day_x', 'file_write_events_x', 'file_open_events_x', 'decoy_file_access_per_day_x', 'web_visits_per_day_x', 'unique_domains_per_day_x', 'unique_paths_per_day_x', 'suspicious_domains_per_day_x', 'file_hosting_visits_per_day_x', 'suspicious_protocol_visits_per_day_x', 'visits_out_of_hours_per_day_x', 'logins_per_day_x', 'unique_pcs_per_day_y_x', 'logins_out_of_hours_x', 'wee